# Terrence Thesis - Case Study
***(Weekly Stress Period) Time Frame 2005-2014--- Changed grid size***

* The time step has been changed from daily to weekly
* The pumping rates were set as a calibration parameter 
+ When the pumping rate was set as the pumping rate of **well 6**, the results were not good as the results by using **average pumping rate of well 5 and 6**
+ The initial temperature distribution has been redefined. 
+ The heat transport parameters (e.g. **Retardation factor / Thermal diffusivity**) still need to be calibrated.

### Import Statements

In [1]:
import flopy
import flopy.modflow as mf
import flopy.utils as fu
import flopy.utils.binaryfile as bf
import matplotlib as mpl 
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import platform
import shutil
import sys
import datetime
import math
from sklearn.metrics import mean_squared_error as mse
import flopy.mt3d as mt3
from flopy.export import vtk
%matplotlib inline

### Setup folder where modflow-files are located

In [2]:
workspace = os.path.join('mf-data-weekly-2011')

In [3]:
if os.path.exists(workspace):
    shutil.rmtree(workspace)

if not os.path.exists(workspace):
    os.makedirs(workspace)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'mf-data-weekly-2011\\model.hds'

### Print versions

In [ ]:
print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

### Setup MF-Package

In [ ]:
modelname = 'model'
exe_name = 'mf2005'

if platform.system() == 'Windows':
    exe_name += '.exe'

In [ ]:
tm = mf.Modflow(modelname=modelname, exe_name=exe_name, version='mf2005', model_ws=workspace)

### Setup Discretization Package

In [ ]:
nrow = 1
ncol = 95
nlay = 10-1

Lx = 950
Ly = 1

botm = np.ones ( (nlay, nrow, ncol), dtype=np.int32 )
top = 33 * np.ones ( (nrow, ncol), dtype=np.int32 )

# Range does not include upper bound !!!
for i in range ( 30, 33):
    top[ :, i ] = 31
for i in range ( 47, 48 ):
    top[ :, i ] = 28
for i in range ( 50, 65):
    top[ :, i ] = 32

# N1 layer
botm[ 0, :, : ] = 28
for i in range ( 30, 33):
    botm[ 0,:, i ] = 25
for i in range ( 50, 65):
    botm[ 0,:, i ] = 25
## N2 layer
#for i in range ( 0, 46):
#    botm[ 1, :, i ] = 24
#for i in range ( 46, 48 ):
#    botm[ 1, :, i ] = 25
#for i in range ( 48, 67 ):
#    botm[ 1, :, i ] = 26
#for i in range ( 67, ncol ):
#    botm[ 1, :, i ] = 25
# N3 layer
for i in range ( 0, 33 ):
    botm[ 1, :, i ] = 20
for i in range ( 33, 46 ):
    botm[ 1, :, i ] = 21.5
for i in range ( 46, 48 ):
    botm[ 1, :, i ] = 24
for i in range ( 48, ncol ):
    botm[ 1, :, i ] = 24.5
# N4 layer
for i in range ( 0, 47 ):
    botm[ 2, :, i ] = 17
for i in range ( 47, 50 ):
    botm[ 2, :, i ] = 16
for i in range ( 50, 60 ):
    botm[ 2, :, i ] = 15
for i in range ( 60, ncol ):
    botm[ 2, :, i ] = 20
# N5 layer
for i in range ( 0, 50 ):
    botm[ 3, :, i ] = 15
for i in range ( 50, 60 ):
    botm[ 3, :, i ] = 14
for i in range ( 60, ncol ):
    botm[ 3, :, i ] = 16
# N6 layer
for i in range ( 0, 60 ):
    botm[ 4, :, i ] = 13
for i in range ( 60, ncol ):
    botm[ 4, :, i ] = 15
# 7 layer
botm[ 5, :, : ] = 10
# 8 layer
botm[ 6, :, : ] = 9
# 9 layer
botm[ 7, :, : ] = -12.5
# 10 layer
botm[ 8, :, : ] = -30

delr = Lx / ncol
delc = Ly / nrow
nper = 505

perlen = np.ones ( nper) * 7
perlen[0] = 1
#nstp = np.ones ( nper) * 7
#nstp[0] = 1
#print(perlen)
steady = np.full ( nper, False, dtype=bool )
steady[ 0 ] = True
dis = mf.ModflowDis(
    tm, 
    nlay=nlay,
    nrow=nrow, 
    ncol=ncol, 
    delr=delr, 
    delc=delc, 
    top=top, 
    botm=botm, 
    nper=nper,
    perlen = perlen,
    steady=steady,
    lenuni = 2,
    xul = 781658.203,#375003.097,
    yul = 5831682.381,#6902348.682,
    proj4_str = "EPSG:32632",
    start_datetime = '11/05/2005'
)

modelxsect= flopy.plot.ModelCrossSection(model=tm, line={'Row':0})
linecollection= modelxsect.plot_grid()

### Setup Basic-Package

Specify boundary conditions and starting heads.

We can set one value for all cells.

In [ ]:
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)

for i in range ( 1, nlay ):
    ibound[ i, :, 0 ] = -1
    ibound[ i, :, -1 ] = -1

for k in range ( 47, 49 ):
    ibound[ 0, :, k ] = 0

strt = np.ones ( (nlay, nrow, ncol), dtype=np.int32 ) * 28
#strt[ 1, :, 1 ] = 28 
#strt[ 1, :, -1 ] = 28

for i in range ( 30, 33 ):
    strt[ 0, :, i ] = 29
for j in range ( 50, 65 ):
    strt[ 0, :, j ] = 29
    
bas = mf.ModflowBas(tm, ibound=ibound, strt=strt)

#bas.plot()

### Setup Constant Head Boundaries

In [ ]:
csv = pd.read_csv('SPA101-weekly.csv', delimiter=' ')
chd_values = csv.iloc[ :, 1 ].values

#chd for each cell
#[lay, row, col, shead, ehead]
spchd = {}
spchd[0] = [
    [1, 0, 0, 28, 28 ], 
    [1, 0, ncol-1, 28, 28 ]
]

for layer in range (2, nlay):
    spchd[0].append ([layer, 0, 0, 28, 28 ])

for stress_period in range ( 1, nper ):
    spchd[stress_period] = [[1, 0, 0, chd_values[stress_period - 1], chd_values[stress_period]]]
    for layer in range (2, nlay):
        spchd[stress_period].append ([layer, 0, 0, chd_values[stress_period-1], chd_values[stress_period]])

chd = mf.ModflowChd(tm, stress_period_data=spchd)
#chd.plot()

### Setup General Head Boundaries

In [ ]:
sp = {}
sp[0] = []

bas12_data = pd.read_csv ( 'Basin1&2-weekly.csv', delimiter=' ' ).iloc[ :, 1 ].values

    
bas3_data = pd.read_csv ( 'Basin3-weekly.csv', delimiter=' ' ).iloc[ :, 1 ].values

for col in range ( 30, 33 ):
    sp[ 0 ].append ( [ 0, 0, col,31, 30] )
    
for col in range ( 50, 65 ):
    sp[ 0 ].append ( [ 0, 0, col, 30, 10] )
    
for stress_period in range ( 1,  nper ):
    sp[ stress_period ] = [ ]
    for col in range ( 30, 33 ):
        if bas3_data[ stress_period - 1 ] == 0:
            sp[ stress_period ].append ( [ 0, 0, col, 28 , 2] )
        else:
            sp[ stress_period ].append ( [ 0, 0, col, 30 + 3  * bas3_data[ stress_period - 1 ] , 30 ] ) 
    for col in range ( 50, 65 ):
        if bas12_data[ stress_period - 1 ] == 0:
            sp[ stress_period ].append ( [ 0, 0, col, 29 , 0.8])
        else:
            sp[ stress_period ].append ( [ 0, 0, col, 30 + 4 * bas12_data[ stress_period - 1 ], 10] )
ghb = mf.ModflowGhb (tm,
                     stress_period_data=sp)
#ghb.plot()

### Setup Wells

In [ ]:
# Pumping Well
wel_data = pd.read_csv('Pumping_well-weekly.csv', delimiter=' ').iloc[:, -2].values 

# -1 pumping rate of well 6
# -2 average pumping rate of well 5 and well 6

spwell = {}
spwell[0] = [7, 0, 41, 0]
for i in range(1, nper):
    spwell[i] = [7, 0, 41, -wel_data[i-1]/70]
    
wel = mf.ModflowWel(tm, stress_period_data=spwell, ipakcb=53)

#wel.plot()

### Setup Soil properties and Flow-Package

In [ ]:
k1 = 2  # fine sand 1.728-7.776
k2 = 19  # middle sand 7.776-34.56
k3 = 49  # stone 34.56-103.68
k4 = 1   # clay
k5 = k2  # middle sand2
k6 = k1  # fine sand 1.728-7.776

# Set default hk-value value to k2
hk = np.ones((nlay, nrow, ncol)) * k2

# N1 layer
hk[0, :, :] = k1  # fine sand

# N2 layer
#hk[1, :, :] = k2  # middle sand

# N3 layer
for i in range( 30, 60 ):
    hk[1, :, i] = k2  
for i in range( 55, 67 ):
    hk[1, :, i] = k1  # clay

#  N4 layer
hk[2, :, :] = k5  # middle sand2

# N5 layer
for i in range( 30, 44 ):
    hk[3, :, i] = k3  # sand & stone
for i in range( 44, 48 ):
    hk[3, :, i] = k4  # clay
for i in range( 48, 60 ):
    hk[3, :, i] = k3  # sand & stone
for i in range( 60, 65 ):
    hk[3, :, i] = k4  # clay

# N6 layer
hk[4, :, :] = k3
for i in range(36, 50 ):
    hk[4, :, i] = k6  # fine sand
for i in range( 50, 60 ):
    hk[4, :, i] = k3  # sand & stone
for i in range( 60, 65 ):
    hk[4, :, i] = k4  # clay

# 7 layer
hk[5, :, :] = k5  # middle sand2

# 8 layer
hk[6, :, :] = k5
for i in range(36 , 50 ):
    hk[6, :, i] = k6  # fine sand

# 9 layer
hk[7, :, :] = k5  # middle sand2

# 10 layer
hk[8, :, :] = k5  # middle sand2


# vertical HK
vk1 = k1*0.3
vk2 = k2*0.3
vk3 = k3*0.3
vk4 = k4*0.3
vk5 = k5*0.3
vk6 = k6*0.3

vka = np.ones((nlay, nrow, ncol), dtype=int) * vk2

# N1 layer
vka[0, :, :] = vk1  # fine sand


# N2 layer
#vka[1, :, :] = vk2  # middle sand

# N3 layer
#vka[2, :, :] = vk3
for i in range( 30, 55 ):
    vka[1, :, i] = vk3  # sand & stone
for i in range( 55, 67 ):
    vka[1, :, i] = vk4  # clay

# N4 layer
vka[2, :, :] = vk5  # middle sand2

# N5 layer
for i in range( 30, 44 ):
    vka[3, :, i] = vk3  # sand & stone
for i in range( 44, 48 ):
    vka[3, :, i] = vk4  # clay
for i in range( 48, 60 ):
    vka[3, :, i] = vk3  # sand & stone
for i in range( 60, 65 ):
    vka[3, :, i] = vk4  # clay

# N6 layer
vka[4, :, :] = vk3
for i in range( 36, 50 ):
    vka[4, :, i] = vk6  # fine sand
for i in range( 50, 60 ):
    vka[4, :, i] = vk3  # sand & stone
for i in range( 60, 65 ):
    vka[4, :, i] = vk4  # clay

# N7 layer
vka[5, :, :] = vk5  # middle sand2

# N8 layer
vka[6, :, :] = vk5
for i in range( 36, 50):
    vka[6, :, i] = vk6  # fine sand

# N9 layer
vka[7, :, :] = vk5  # middle sand2

# N10 layer
vka[8, :, :] = vk5  # middle sand2

laytyp = [1, 1, 1, 1, 1, 1, 1, 1, 1]

# Specific yield 
sy1 = 0.25 # fine sand 
sy2 = 0.35 # middle sand
sy3 = 0.35 # stone 
sy4 = 0.16 # clay
sy5 = 0.35 # middle sand
sy6 = 0.25 # fine sand 

sy = np.ones((nlay, nrow, ncol)) * sy2

# N1 layer
sy[0, :, :] = sy1  # fine sand


# N2 layer
#-1sy[1, :, :] = sy2  # middle sand

# N3 layer
#sy[2-1, :, :] = sy3
for i in range( 30, 55 ):
    sy[2-1, :, i] = sy3  # sand & stone
for i in range( 55, 67 ):
    sy[2-1, :, i] = sy4  # clay

# N4 layer
sy[3-1, :, :] = sy5  # middle sand2

# N5 layer
for i in range( 30, 44 ):
    sy[4-1, :, i] = sy3  # sand & stone
for i in range( 44, 48 ):
    sy[4-1, :, i] = sy4  # clay
for i in range( 48, 60 ):
    sy[4-1, :, i] = sy3  # sand & stone
for i in range( 60, 65 ):
    sy[4-1, :, i] = sy4  # clay

# N6 layer
sy[5-1, :, :] = sy3
for i in range( 36, 50 ):
    sy[5-1, :, i] = sy6  # fine sand
for i in range( 50, 60 ):
    sy[5-1, :, i] = sy3  # sand & stone
for i in range( 60, 65 ):
    sy[5-1, :, i] = sy4  # clay

# N7 layer
sy[6-1, :, :] = sy5  # middle sand2

# N8 layer
sy[7-1, :, :] = sy5
for i in range( 36, 50):
    sy[7-1, :, i] = sy6  # fine sand

# N9 layer
sy[8-1, :, :] = sy5  # middle sand2

# N10 layer
sy[9-1, :, :] = sy5  # middle sand2
ss = 1e-5

lpf = mf.ModflowLpf(tm, laytyp=laytyp, hk=hk, layvka=0, vka=vka, sy= 0.35 ,  ss=ss, ipakcb=53)

#lpf.plot()

### Setup Solver Package and Output-control with defaults

In [ ]:
budgetfile = '{}.cbc'.format(modelname)
budget_filerecord = [budgetfile]
oc = mf.ModflowOc(tm, 
                   save_every=True,
                   save_types=[ 'save head', 'save budget'],
                   budget_filerecord=budget_filerecord,
                   compact=True
                  )


pcg = mf.ModflowPcg (tm, rclose=1e-1)

lmt = flopy.modflow.ModflowLmt ( tm, output_file_name = modelname + '.ftl', output_file_header='extended',
                                 output_file_format='formatted')

### Write Model input and run the model

In [ ]:
tm.write_input()
success = tm.run_model()#silent = True)
#cbc = bf.CellBudgetFile(os.path.join(workspace, modelname + '.cbc'))
flopy.export.shapefile_utils.model_attributes_to_shapefile('model.shp', tm, package_names = ['dis'])


### Read and Plot Head Observations

In [ ]:
headobj = bf.HeadFile(os.path.join(workspace, modelname + '.hds'))

#observation wells

obs = [(1, 0, 65),
       (1, 0, 48),
       (1, 0, 46)]
name = ['SPA157','SPA168','SPA165']

for j in range (0,3):
    ts = headobj.get_ts ( idx= obs[j] )
    np.savetxt ( "HD.csv", ts, delimiter=',', header='Days,Head' )
    model = pd.read_csv ( 'HD.csv',delimiter=',')
    date = []
    for d in model['# Days']:
        date.append(datetime.datetime(2005,5,11)+datetime.timedelta(d-1)) 
    model['date'] = date
    model['nweek'] = np.int64((model['# Days']-1)/7+1)

    #observation = pd.read_csv ( name[j]+'.csv' , delimiter=',' ,nrows = 150)
    #date = []
    #for d in observation['# Days']:
    #    date.append(datetime.datetime(2005,5,11)+datetime.timedelta(d-1)) 
    #observation['date'] = date
   
    
    observation1 = pd.read_csv ( name[j]+'-weekly.csv' , delimiter=' ', header = -1)
    date = []
    for d in observation1.iloc[:,0]:
        date.append(datetime.datetime(2005,5,11)+datetime.timedelta((d-1)*7)) 
    observation1['date'] = date    
    #observation1['nweek'] = observation1.iloc[:,0]
    observation1.columns = ['nweek','Head','date']

    #print(observation1)
    plt.plot ( model['date'], model.iloc[:,1], label='model' , c = 'slategray')
    #plt.plot( observation['date'], observation.iloc[:,1], '.', label='Observation', markersize = 2,c='darkcyan' )
    plt.plot( observation1['date'], observation1.iloc[:,1], '.', label='Observation1', markersize = 2 )
    plt.legend ( [ "model", "Observation" ] )
    plt.xlabel ( 'Date' )
    plt.ylabel ( 'Groundwater Level (m)' )
    plt.title ( name[j] , loc='right' )
    plt.show ()
    
    merge = pd.merge ( model, observation1,left_on='date',right_on='date',how='inner')
    merge = merge.iloc[:,[2,1,-1]]
    merge.columns = ['date','Head_model','Head_obs']
    
    h1 = merge['Head_model']
    h2 = merge['Head_obs']
    rmse = math.sqrt ( mse ( h1, h2 ) )
    nrmse = rmse/(np.max(h2)-np.min(h2))
    rmse = '{:01.2f}'.format(rmse)
    nrmse = '{:01.2f}'.format(nrmse)
    u = []
    u = h1-h2
    a = h1 - h1
    plt.plot(merge['date'],a,c='slategray',label='Standard')
    plt.plot(merge['date'],u,'.',label='Differences',markersize = 2,c = 'darkcyan')
    plt.legend(['Standard','Differences'])
    plt.xlabel('Date',horizontalalignment='right',x=1.0)
    plt.ylabel('Errors (model - observation) (m)')
    plt.title( name[j] + '(NRMSE = ' + str ( nrmse )+')', loc='right')
    plt.show()
    
    x = [int(min(h2)),32]
    y = [int(min(h2)),32]
    plt.scatter(h1,h2,s = 5,c='slategray')
    plt.plot(x,y,c='darkcyan')
    plt.ylim(int(min(h2)),32)
    plt.xlim(int(min(h2)),32)
    plt.xlabel('Model results (m)')
    plt.ylabel('Observation values (m)')
    plt.subplot().spines['right'].set_color('none')
    plt.subplot().spines['top'].set_color('none')
    plt.subplot()
    plt.title(name[j] + '(RMSE = ' + str ( rmse )+')', loc='right')
    plt.show()
    
    #os.remove('HD.csv')


### MT3DMS 

In [ ]:
if os.path.isfile('Basin-T-weekly.csv'):
    heat = pd.read_csv('Basin-T-weekly.csv',delimiter=' ',header = -1)
else:
    os.chdir("..")
    heat = pd.read_csv('Basin-T-weekly.csv',delimiter=' ',header = -1)

heat_basin = heat.iloc[:,-1].values
#print (heat, heat_basin)
mt = flopy.mt3d.Mt3dms(modelname = 'heat',version='mt3dms',  exe_name='mt3dms5b', 
                       model_ws = workspace, 
                       modflowmodel = tm, ftlfilename = modelname + ".ftl", verbose = True)

adv = mt3.Mt3dAdv(mt, mixelm = -1, percel = 0.75)
icbund = np.ones((nlay, nrow, ncol), dtype=np.int32)
icbund[:,:,0] = -1
icbund[:,:,-1] = -1
sconc_1 = np.ones((nlay, nrow, ncol)) * 10

for i in range ( 30, 55):
    sconc_1[ :, :, i ] = 18
for i in range ( 38, 45):
    sconc_1[ :, :, i ] = 13.5
for i in range ( 30, 33):
    sconc_1[ 0, :, i] = 12
for i in range ( 50, 65):
    sconc_1[ 0, :, i] = 12
    
obs = [ ( 7, 0, 41)]

btn = mt3.Mt3dBtn(mt,icbund= icbund, ncomp=1,mcomp=0, sconc = sconc_1, prsity = sy, species_names=['heat'], obs=obs,
                nprobs=7)

dsp = mt3.Mt3dDsp(mt, al = 2, trpt= 0.1,trpv = 0.03, dmcoef= 0.13)

gcg = mt3.Mt3dGcg(mt, mxiter=1,iter1=50,isolve = 2,cclose = 1e-2,ncrs =1)             
rct = mt3.Mt3dRct(mt,isothm=1,ireact=0, prsity2= sy, igetsc=0, rhob = 1.7e3, sp1= 0.25e-3)

ssm_data = {}
ssm_data[0] = []

itype = flopy.mt3d.Mt3dSsm.itype_dict()

for i in range( 30, 33):
    ssm_data[0] = ssm_data[0]+ [[0, 0, i, 14 , itype[ 'GHB' ]]]
for i in range ( 50, 65):
    ssm_data[0].append( [ 0, 0, i, 14, itype[ 'GHB' ] ] )
    
for j in range (1,280):
    ssm_data[ j ] = [ ]
    for i in range ( 30, 33):
         ssm_data[ j ].append ( [ 0, 0, i, heat_basin[ j - 1 ], itype[ 'GHB' ] ] )
    for i in range ( 50, 65):
         ssm_data[ j ].append ( [ 0, 0, i, heat_basin[ j - 1 ], itype[ 'GHB' ] ] )
for j in range(280,nper):
    ssm_data[ j ] = [ ]
    for i in range ( 30, 33):
        ssm_data[ j ].append ( [ 0, 0, i, 14, itype[ 'GHB' ] ] )
    for i in range ( 50, 65):
        ssm_data[ j ].append ( [ 0, 0, i, 14, itype[ 'GHB' ] ] )

ssm = flopy.mt3d.Mt3dSsm(mt, stress_period_data=ssm_data)

mt.write_input()
mt.run_model(silent = True)

if os.path.isfile('N5_T-weekly.csv'):
    n = pd.read_csv('N5_T-weekly.csv',delimiter=',',header=0, index_col = 0)
    w = pd.read_csv('N6_T-weekly.csv',delimiter=',',header=0, index_col = 0)
else:
    os.chdir("..")
    n = pd.read_csv('N5_T-weekly.csv',delimiter=',',header=0, index_col = 0)
    w = pd.read_csv('N6_T-weekly.csv',delimiter=',',header=0 ,index_col = 0)
if os.path.isfile('MT3D001.OBS'):
    t = pd.read_csv('MT3D001.OBS',delimiter=' ', nrows= 600,skiprows=[0,1],skipinitialspace=True,skip_blank_lines=True,usecols=[1,2])

else:
    os.chdir(workspace)
    t = pd.read_csv('MT3D001.OBS',delimiter=' ', nrows= 600,skiprows=[0,1],skipinitialspace=True,skip_blank_lines=True,usecols=[1,2])

t1 = t.iloc[:,0].values
y1 = []
for i in t1:
    y1.append([datetime.date(2005,5,11)+datetime.timedelta(i-1)])
t2 = t.iloc[:,1].values

#print(t,n,w)

n1 = n['days']
y2 = []
for i in n1:
    y2.append([datetime.date(2005,5,11)+datetime.timedelta(np.float64(i-1))])
n2 = n.iloc[:,0].values

w1 = w['days']
y3 = []
for i in w1:
    y3.append([datetime.date(2005,5,11)+datetime.timedelta(np.float64(i-1))])
w2 = w.iloc[:,0].values

plt.plot(y1,t2,c='olive')
plt.scatter(y2,n2,c='orange',s=2)
plt.scatter(y3,w2,c='springgreen',s=2)
plt.legend(['model','ObservationN5','ObservationN6'])
plt.xlabel ( 'Date' )
plt.ylabel ( 'Temperature (°C)' )
plt.title('Temperature Simulation')
#plt.savefig('temperature.png')
plt.show()
plt.close()

x = pd.merge ( t, n,left_on= t.iloc[:,0].values,right_on='days',how='inner')

v1 = x.iloc[:,1].values
v2 = x.iloc[:,2].values
x1 = [7,18.5]
y1 = [7,18.5]
plt.scatter(v1,v2,s = 5,c='slategray')
plt.plot(x1,y1,c='darkcyan')
plt.ylim(7,18.5)
plt.xlim(7,18.5)
plt.xlabel ( 'Model Results' )
plt.ylabel ( 'Observation Temperature (Well NO.5)' )
rmse =  math.sqrt ( mse ( v1, v2 ) )
rmse = '{:01.2f}'.format(rmse)
plt.title('Well NO.5 (RMSE = ' + str (rmse)+')')
plt.show()
plt.close()


x = pd.merge ( t, w,left_on=t.iloc[:,0].values,right_on='days',how='inner')
v1 = x.iloc[:,1].values
v2 = x.iloc[:,2].values
x = [7,18.5]
y = [7,18.5]
plt.scatter(v1,v2,s = 5,c='slategray')
plt.plot(x,y,c='darkcyan')
plt.ylim(7,18.5)
plt.xlim(7,18.5)
plt.xlabel ( 'Model Results' )
plt.ylabel ( 'Observation Temperature (Well NO.6)' )
rmse =  math.sqrt ( mse ( v1, v2 ) )
rmse = '{:01.2f}'.format(rmse)
plt.title('Well NO.6 (RMSE = ' + str (rmse)+')')
plt.show()
plt.close()